In [45]:
import pandas as pd
import numpy as np
import json

In [66]:
metadata = pd.read_csv('../data/output/raw_metadata.csv')
metadata.head()

,composer,details_url,duration,filename_xml,genre,hasError,key,name,primary_time_signature,secondary_time_signatures,source_url,tempo,year,filename,filename_mid,filename_abc
0,A. Adam,piece-info.cgi?id=897,72.000000,../data/output/xml/6e355a07d5531f372a2f7fb9ac0...,Classical,False,G major,Giselle - Pas de deux (1er Acte),4/4,[],NaN,80,,6e355a07d5531f372a2f7fb9ac084a120c34440a.mid,../data/output/mid/6e355a07d5531f372a2f7fb9ac0...,../data/output/abc/6e355a07d5531f372a2f7fb9ac0...
1,I. M. F. Albéniz,piece-info.cgi?id=898,262.000000,../data/output/xml/cf0e37cd297b4b030a506f1e1e2...,Romantic,False,d minor,Rumores de la Caleta,4/4,[],NaN,60,1887,cf0e37cd297b4b030a506f1e1e2c637352dcc12d.mid,../data/output/mid/cf0e37cd297b4b030a506f1e1e2...,../data/output/abc/cf0e37cd297b4b030a506f1e1e2...
2,J. André,piece-info.cgi?id=207,95.769231,../data/output/xml/9235e4d8f77349089d45f475fb8...,Classical,False,C major,Sonatine,4/4,[],NaN,104,18th Century,9235e4d8f77349089d45f475fb852ab71c954de8.mid,../data/output/mid/9235e4d8f77349089d45f475fb8...,../data/output/abc/9235e4d8f77349089d45f475fb8...
3,Anonymous,piece-info.cgi?id=679,33.626374,../data/output/xml/6ece453ca4108e48ea6e6d0fec6...,Popular / Dance,False,D major,La Native,4/4,[],NaN,91,c1860,6ece453ca4108e48ea6e6d0fec6996441adc09b5.mid,../data/output/mid/6ece453ca4108e48ea6e6d0fec6...,../data/output/abc/6ece453ca4108e48ea6e6d0fec6...
4,Anonymous,piece-info.cgi?id=680,41.860465,../data/output/xml/96b2c5072caf99d739a42bad844...,Popular / Dance,False,G major,Les Graces,4/4,[],NaN,86,c1860,96b2c5072caf99d739a42bad844e11b6003445aa.mid,../data/output/mid/96b2c5072caf99d739a42bad844...,../data/output/abc/96b2c5072caf99d739a42bad844...


In [6]:
len(metadata.index)

2187

In [7]:
# Carl Philipp Emanuel Bach
metadata.composer[metadata['composer'] == 'C. P. E. Bach'] = 'cpe_bach'

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [27]:
import re
from unidecode import unidecode

def remove_initials(string):
    return re.sub('\s*\w\.\s*', '', string)

def normalize_composer_name(composer_name):
    try:
        composer_name = remove_initials(composer_name)
        composer_name = unidecode(composer_name)
        composer_name = composer_name.lower()
        return composer_name.split(',')[0]
    except:
        print('Bad composer name: ' + str(composer_name))
        return 'Unknown'

In [28]:
metadata.ix[562, "Composer"] = "Chopin"

In [29]:
metadata['composer'] = metadata['composer'].map(normalize_composer_name)

Bad composer name: None


In [30]:
composers = metadata['composer'].unique()

In [31]:
composer_name_regex = '|'.join(composers)
metadata['name'][metadata['name'].str.contains(composer_name_regex, case=False)]

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


395                              Fugue sur le nom de Bach
562           N6e. Chopin, Mazurka, op. 41 no. 2, mm. 1-8
961                      Suite española\n(1886)\n Sevilla
965     53 Studies on Études on Frédéric Chopin\n(1914...
973     M. P. Heller: Joy of Christmas\n(Christmas Fan...
977     Robert Leonard: Christmas Fantasia, Opus 99 Ch...
981     Lyric Pieces Book III, Opus 43\n(1886)\n No. 2...
1001        Danzas españolas\n(1900)\n No. 4 - Villanesca
1026    The Seasons, Opus 37a\n(1876)\n December - Chr...
1165                        Grieg Lyrische Stcke Op-43 Nr
1256                                 Tango-(Arr Godowsky)
1357                                                 s901
1530                           SAENS-BEETHOVEN-Variations
1666                   Liebeslied-(transcribed by LISZT-)
1801                  HAYDN-Variations-(4 tracks GM file)
1809                 Scherzo-from Symph No-9 (Arr LISZT-)
1839                                                 s902
1884          

In [51]:
metadata['composer'].unique()
metadata['composer'].

array(['adam', 'albeniz', 'andre', 'anonymous', 'cpe_bach', 'bach',
       'beethoven', 'benda', 'bizet', 'blumenthal', 'brahms', 'brown',
       'burgmuller', 'chopin', 'czerny', 'comeau', 'debussy', 'diabelli',
       'doonan', 'fischer', 'streabbog (gobbaerts)', 'grieg', 'handel',
       'haydn', 'iliev', 'hunter', 'joplin', 'kruetzer', 'kumar', 'kuhlau',
       'liszt', 'magnenat', 'mendelssohn-bartholdy', 'minkus', 'mozart',
       'pease', 'pachelbel', 'muller', 'pejacsevich', 'pleyel',
       'mussorgsky', 'rachmaninoff', 'rameau', 'rimsky-korsakov', 'satie',
       'scarlatti', 'schubert', 'schumann', 'scriabin', 'sidwell', 'sousa',
       'spagnoletti', 'stanchinsky', 'strauss j', 'tchaikovsky',
       'traditional', 'turpin', 'verdi', 'wanhal', 'yaniewicz', 'Unknown',
       'clementi', 'borodin', 'burgmueller', 'balakirew', 'godowsky',
       'christmas', 'granados', 'sinding', 'ravel', 'rachmaninov',
       'moszkowski', 'mendelssohn', 'skriabin', 'mozartart', 'faure',
    

In [32]:
def extract_year_from_title(title_string):
    match = re.search('\((\d{4})\)', title_string)
    if match:
        return int(match.group(1))
    
    return None

In [33]:
extract_year_from_title('Sonatina Opus 36 No. 1, Opus 36\n(1797)\n')

1797

In [190]:
#metadata['year'] = pd.read_csv('all_metadata.csv')['year']

In [34]:
years_from_title = metadata['name'].map(extract_year_from_title)

for index, row in metadata[years_from_title.notnull() & metadata['year'].notnull()].iterrows():
    raise Exception('Warning! Year present both in name ({0}) and in metadata ({1})'.format(row['name'], row['year']))

metadata['year'] = metadata['year'].fillna(years_from_title)

In [35]:
def extract_year(year_obj):
    
    if not year_obj or str(year_obj) == 'nan':
        return None, False
    
    try:
        int_year = int(year_obj)
        return int_year, True
    except Exception:
        pass
    
    try:
        int_year = math.floor(year_obj)
        return int_year, True
    except Exception:
        pass
    
    match = re.search('(~|c|around|c.)?\s*(\d{4})\??', year_obj) # year with ~ or ?
    if match:
        return int(match.groups()[1]), False
    
    match = re.search('(\d{4})-\d{1,4}', year_obj) # year period. Return lower bound
    if match:
        return int(match.groups()[0]), False
    
    match = re.search('.+-(\d{4})', year_obj) # year period. Return lower bound
    if match:
        return int(match.groups()[0]), False
    
    
    match = re.search('(\d{4})/\w{2,4}', year_obj) # exact date
    if match:
        return int(match.groups()[0]), True
    
    match = re.search('\w{3,4}\.*(\d{4})', year_obj) # month and year
    if match:
        return int(match.groups()[0]), True
    
    match = re.search('(\d{2})th', year_obj) # century
    if match:
        return int(match.groups()[0]) * 100 - 50, False
    
    return None, False

In [36]:
extracted_years = metadata['year'].map(extract_year)

# todo: there should be a better way...
metadata['year'] = pd.to_numeric(extracted_years.map(lambda x: x[0]))
metadata['year_exact'] = extracted_years.map(lambda x: x[1])

In [38]:
from collections import namedtuple

Period = namedtuple('Period', ['start', 'end'])

music_periods = { # this is not exactly exact. MUST DOUBLE CHECK AFTER
    'baroque': Period(1600, 1750), 
    'classicism': Period(1750, 1820), 
    'romanticism': Period(1820, 1890), 
    'modern': Period(1890, 1950),
    'folk': Period(1000, 1600),
    'jazz': Period(0, 0) # sorry jazz. i love you it is just not your day
}

metadata['period'] = metadata['genre'].map(lambda x: str(x).lower() if x else None)
metadata[~metadata['period'].isin(music_periods.keys())]['period'] = None
metadata[metadata['period'] == 'romantic']['genre'] = 'romanticism'

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [39]:
def get_form_from_title(title_string):
    if not title_string:
        return None
    
    title_string = unidecode(title_string)
    for form in musical_forms:
        if re.search(form, title_string, re.IGNORECASE):
            return form
    
    return None

In [40]:
musical_forms = [
    'rag',
    'sonata',
    'sonatina',
    'rhapsod',
    'suite',
    'canon',
    'nocturne',
    'valse',
    'ballad',
    'polonaise',
    'fantasia',
    'song', 
    'march', 
    'technique', 
    'etude', 
    'mazurka', 
    'fugue', 
    'rondo',
    'prelude']

metadata['musical_form'] = metadata['genre'].map(lambda x: get_form_from_title(str(x)))

In [41]:
metadata['musical_form'] = metadata['musical_form'].fillna(metadata['name'].map(get_form_from_title))

In [42]:
def get_period_by_year(year):
    if not year or year is None or str(year) == 'nan':
        return None
    
    for period_name, period_dates in music_periods.items():
        if period_dates.start <= year <= period_dates.end:
            return period_name
        
    return None

In [43]:
metadata['period'] = metadata['period'].fillna(metadata['year'].map(get_period_by_year))

In [46]:
composers = json.load(open('composers.json', 'r'))

In [58]:
metadata[metadata['period'] == 'nan']['period'] = None

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [59]:
composer_periods = metadata['composer'].map(lambda x: composers[x] if x in composers else None)
metadata['period'] = metadata['period'].fillna(composer_periods)

In [60]:
metadata.groupby(by='period')['period'].count()

period
baroque             236
classical           132
classicism           81
folk                 58
jazz                 25
march                 3
modern               14
nan                1230
popular / dance       6
romantic            164
romanticism         216
song                  3
technique            19
Name: period, dtype: int64

In [352]:
del metadata['processed_file_path']
del metadata['raw_file_path']

In [6]:
import os.path
metadata['filename'] = metadata['processed_file_path'].map(lambda x: os.path.basename(str(x)))

KeyError: 'processed_file_path'

In [65]:
metadata.to_csv('../data/output/metadata.csv', index=False)

In [62]:
metadata.groupby(by='primary_time_signature')['primary_time_signature'].count()

primary_time_signature
4/4    2187
Name: primary_time_signature, dtype: int64

In [64]:
metadata.groupby(by='secondary_time_signatures')['secondary_time_signatures'].count()

secondary_time_signatures
[]    2187
Name: secondary_time_signatures, dtype: int64